In [20]:
import github3

In [22]:
import yaml

In [32]:
gh = github3.login(token='???')
org = gh.organization('dd-decaf')

In [61]:
MAPPING = {
    'ANNOTATIONS': 'gene-to-reactions',
    'GENOMICS': 'ice'
}
EXTRA = ['ILOOP_API']

def node_name(repo, key=''):
    repo = MAPPING.get(repo, repo)
    repo += ' ' + key
    repo = repo.replace('-', ' ')
    return ''.join(repo.title().split())

final = {}
for repo in org.iter_repos():
    compose = repo.contents('docker-compose.yml')
    if compose:
        yml = yaml.load(compose.decoded)
        if 'version' in yml:
            yml = yml['services']
        for k, v in yml.items():
            new_links = []
            for en in v.get('environment', []):
                e = en.split('=')[0]
                if e.endswith('_API') and e not in EXTRA:
                    new_links.append(node_name(e.replace('_API', '').replace('_', '-'), 'web'))
            if 'links' in v:
                v['links'] = [node_name(repo.name, s) for s in v['links']]
            else:
                v['links'] = []
            v['links'].extend(new_links)
            final[node_name(repo.name, k)] = v
with open('docker-compose.yml', 'w') as f:
    f.write(yaml.dump(final, default_flow_style=False))

In [80]:
for repo in org.iter_repos():
    req = repo.contents('requirements.txt')
    if req:
        print('-----')
        print(repo.name)
        print('-----')
        for line in req.decoded.decode('utf8').split():
            package = line.split('==')[0]
            r = requests.get('http://pypi.python.org/pypi/{}/json'.format(package))
            if r.status_code == 200:
                print(package, r.json()['info']['license'])

-----
gene-to-reactions
-----
venom MIT
requests Apache 2.0
redis MIT
gnomic Apache
codecov http://www.apache.org/licenses/LICENSE-2.0
grpcio-tools Apache License 2.0
nose GNU LGPL
-----
model
-----
aiohttp Apache 2
aiozmq BSD
aiohttp_cors Apache License, Version 2.0
msgpack-python UNKNOWN
cobra LGPL/GPL v2+
cameo Apache License Version 2.0
swiglpk GPL v3
gnomic Apache
requests Apache 2.0
redis MIT
codecov http://www.apache.org/licenses/LICENSE-2.0
gunicorn MIT
uvloop MIT
venom MIT
grpcio Apache License 2.0
pytest-asyncio Apache 2.0
pytest-cov MIT
aioredis MIT
deepdiff MIT
tqdm MPLv2.0, MIT Licences
-----
iloop-to-model
-----
aiohttp Apache 2
aiozmq BSD
aiohttp_cors Apache License, Version 2.0
msgpack-python UNKNOWN
requests Apache 2.0
redis MIT
codecov http://www.apache.org/licenses/LICENSE-2.0
gunicorn MIT
uvloop MIT
pytest MIT license
pytest-asyncio Apache 2.0
pytest-cov MIT
-----
camilo
-----
requests Apache 2.0
pandas BSD
-----
upload
-----
aiohttp Apache 2
aiozmq BSD
aiohttp_cors